# 2022-12-09运行

2022-07-24运行

In [1]:
from Bio import Medline
from Bio import Entrez #Entrez查询
import pickle
Entrez.email="357610983@qq.com"

In [2]:
import time
import datetime

In [3]:
now=datetime.datetime.now()

In [4]:
nowtime=now.strftime('%Y/%m/%d')

In [5]:
now_time=now.strftime('%Y_%m_%d')

In [6]:
print(now_time)

2023_03_14


In [7]:
time_without_day=now.strftime('%Y/%m')

In [8]:
time_without_day

'2023/03'

In [9]:
### today 2022_07 FORMAT
### now.strftime('%Y_%m')

In [10]:
### 2022/07/30 FORMAT
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [11]:
### 2022_07_30 FORMAT
def cal_time2(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y_%m_%d')
    return time

In [12]:
### 2022_07 FORMAT
def cal_time3(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y_%m')
    return time

In [13]:
former_time=cal_time(nowtime)

In [14]:
former_time

'2022/12/01'

In [17]:
# former_time='2022/09/01'

In [18]:
terms='("aging"[Title/Abstract] OR "ageing"[Title/Abstract] OR "longevity"[Title/Abstract] OR "centenarian"[Title/Abstract] OR "centenarians"[Title/Abstract] \
OR "the elderly"[Title/Abstract] OR "the aged"[Title/Abstract] OR "old people"[Title/Abstract] OR "older people"[Title/Abstract] OR "old age"[Title/Abstract]  AND \
Journal Article[ptyp] AND "humans"[MeSH Terms] AND English[lang] AND ("'+str(former_time)+'"[Date - Publication] : "3000"[Date - Publication]))'
handle_0 = Entrez.esearch(db="pubmed", term=terms,retmax=1000000)
record = Entrez.read(handle_0)
idlist = record["IdList"]
No_Papers = len(idlist)
print(No_Papers) # 共213篇文献 2019-10-30

9898


In [19]:
import http.client
print(http.client.HTTPConnection._http_vsn)
print(http.client.HTTPConnection._http_vsn_str)

11
HTTP/1.1


import http.client
http.client.HTTPConnection._http_vsn = 10
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

尝试了retmax与retstart组合，连接会中断；尝试了step为10000，有两个下载不完整，故采用5000的方法。

In [20]:
import time

In [21]:
len(idlist)

9898

In [22]:
len(set(idlist))

9898

In [23]:
idlist[0]

'36896533'

In [24]:
res=[]
for i in range(0,len(idlist),10000):
    id=idlist[i:i+10000]
    handle_1 = Entrez.efetch(db="pubmed", id=id, rettype="medline",retmode="text")
    records = Medline.parse(handle_1)
    res.append(list(records))
    # print("%i of records have been completed" % end)

import http.client
http.client.HTTPConnection._http_vsn = 11
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.1'

In [25]:
raw_pick=[]
for i in res:
    raw_pick+=i

In [26]:
len(raw_pick)

9898

raw_pick=pickle.load(open('raw_pick.pkl','rb'))

In [27]:
import pickle

In [28]:
cal_time3(nowtime)

'2022_12'

In [29]:
import pickle
import json

pickle.dump(pmid_list,open('pmid_list','wb'))
pickle.dump(reserved_raw_pick,open('reserved_raw_pick','wb'))
pickle.dump(reserved_raw_pick_pmid,open('reserved_raw_pick_pmid','wb'))
pickle.dump(pmids_list,open('pmids_list','wb'))
pickle.dump(raw_pick,open('raw_pick','wb'))

pmid_list=pickle.load(open('pmid_list','rb'))
reserved_raw_pick=pickle.load(open('reserved_raw_pick','rb'))
reserved_raw_pick_pmid=pickle.load(open('reserved_raw_pick_pmid','rb'))
pmids_list=pickle.load(open('pmids_list','rb'))
raw_pick=pickle.load(open('raw_pick','rb'))

In [30]:
import pymongo 

In [31]:
from pymongo import MongoClient

In [32]:
client = MongoClient(username="hald",password="XXX")

In [33]:
db = client.hald

# pubmed_raw_pmid

In [34]:
collections_pubmed_raw_pmid=db.pubmed_raw_pmid

collections_pubmed_raw_pmid.remove({'addtime':now_time})

In [35]:
pmids=list(collections_pubmed_raw_pmid.find())

In [36]:
pmids[:2]

[{'_id': ObjectId('6393476d5303aaecac1f079a'),
  'PMID': '35796512',
  'addtime': '20220724'},
 {'_id': ObjectId('6393476d5303aaecac1f079b'),
  'PMID': '35796396',
  'addtime': '20220724'}]

In [37]:
pmid_list=[]
for i in pmids:
    pmid_list.append(i.get('PMID'))

In [38]:
pmid_list[:2]

['35796512', '35796396']

In [39]:
len(pmid_list)

287718

In [40]:
pmid_dict={}
for i in pmids:
    pmid_dict[i.get('PMID')]=i.get('PMID')

In [41]:
len(pmid_dict)

287718

In [42]:
len(raw_pick)

9898

In [43]:
now_time

'2023_03_14'

In [45]:
reserved_raw_pick=[]
reserved_raw_pick_pmid=[]
for i in raw_pick:
    if i.get('AB'):
        if not pmid_dict.get(i.get('PMID')):
            i['addtime']=now_time
            reserved_raw_pick.append(i)
            reserved_raw_pick_pmid.append({'PMID':i.get('PMID'),'addtime':i.get('addtime')})

In [46]:
# 6561
len(reserved_raw_pick)

5256

In [47]:
collections_pubmed_raw_pmid.insert_many(reserved_raw_pick_pmid)

In [48]:
len(list(collections_pubmed_raw_pmid.find()))

292974

In [49]:
# 287839
# 287463
# len(list(collections_pubmed_raw_pmid.find()))

# pubmed_raw (reserved_raw_pick)

In [50]:
collections_pubmed_raw = db.pubmed_raw

collections_pubmed_raw.remove({'addtime':now_time})

In [51]:
collections_pubmed_raw.insert_many(reserved_raw_pick)

# pubtator_raw_pmid

In [52]:
collection_pubtator_raw_pmid = db.pubtator_raw_pmid

In [53]:
previous_pubtator_pmid=list(collection_pubtator_raw_pmid.find({}))

In [54]:
len(previous_pubtator_pmid)

265577

In [55]:
previous_pubtator_pmids_dict={}
for i in previous_pubtator_pmid:
    previous_pubtator_pmids_dict[i.get('PMID')]=i.get('PMID')

In [56]:
current_pubtator_pmid=[]
for i in raw_pick:
    if i.get('AB'):
        if not previous_pubtator_pmids_dict.get(i.get('PMID')):
            i['addtime']=now_time
            current_pubtator_pmid.append(i.get('PMID'))

In [57]:
len(current_pubtator_pmid)

5305

In [58]:
pmids_list=[]
for i in current_pubtator_pmid:
    pmids_list.append(i)

In [59]:
len(pmids_list)

5305

In [61]:
import requests

In [62]:
def SubmitPMIDList(pmids,Format,now_time):
	json = {"pmids": pmids}

	r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/"+Format , json = json)
	if r.status_code != 200 :
		print ("[Error]: HTTP code "+ str(r.status_code))
	else:
		with open(now_time+'_pubtator_raw.txt','a',encoding="UTF-8") as fileread:
			fileread.write(str(r.text.encode("utf-8"),encoding="UTF-8"))

In [70]:
import time
pubtator_raw=[]
for i in range(0,len(pmids_list),1000):
    SubmitPMIDList(pmids_list[i:i+1000],'biocjson',now_time)
    time.sleep(0.1)

import time
pubtator_raw=[]
for i in range(0,len(pmids_list),1):
    SubmitPMIDList(pmids_list[i:i+1],'biocjson',now)
    # time.sleep(0.1)

In [71]:
ttt=open(now_time+'_pubtator_raw.txt',"r")

In [72]:
overjson=[]
for l in ttt.readlines():
    overjson.append(l.rstrip())

In [73]:
### 变少的原因包括pubtator还未进行文本挖掘。

In [74]:
# 6494
len(overjson)

5067

In [75]:
import copy

In [76]:
pubtator_list=copy.deepcopy(overjson)

In [77]:
pubtator_json=[]
for i in pubtator_list:
    pubtator_json.append(json.loads(i))

In [78]:
pubtator_pmid_list=[]
for i in pubtator_json:
    pubtator_pmid_list.append(i.get('id'))

In [79]:
# 6307
len(pubtator_pmid_list)

5067

In [80]:
why=[]
for i in pmids_list:
    if i not in pubtator_pmid_list:
        why.append(i)

In [81]:
why[5]

'36722236'

In [82]:
len(why)

238

In [83]:
why[:3]

['36724157', '36722661', '36722644']

In [84]:
pubtator_pmid_list[0]

'36896533'

In [85]:
current_pubtator_pmid_list=[]
for i in pubtator_pmid_list:
    current_pubtator_pmid_list.append({'PMID':i,'addtime':now_time})

In [86]:
len(current_pubtator_pmid_list)

5067

In [87]:
collection_pubtator_raw_pmid.insert_many(current_pubtator_pmid_list)

# pubtator_info  (raw_pick_pubtator)

In [88]:
collections_pubtator_info = db.pubtator_info

collections_pubtator_info.remove({'addtime':now_time})

In [89]:
filtered_pubtator_json=[]
for i in pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                if j.get('infons').get('identifier')=='9606':
                    # if j.get('infons').get('type') =='Gene' :
                    # if j.get('infons').get('type') in ('Gene', 'Chemical', 'Mutation') :
                    filtered_pubtator_json.append(i)
                    break

In [91]:
# 4874
len(filtered_pubtator_json)

3899

In [92]:
second_filtered_pubtator_json=[]
for i in filtered_pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                    if j.get('infons').get('type') in ('Gene', 'Chemical', 'Mutation','Disease') :
                        second_filtered_pubtator_json.append(i)
                        break

In [93]:
# 4257
len(second_filtered_pubtator_json)

3400

In [94]:
entities_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        entities_list=[]
        for j in i.get('passages')[1].get('annotations'):
            entities_list.append(j.get('text'))
        entities_dict[i.get('id')]=entities_list

In [95]:
second_filtered_pubtator_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        second_filtered_pubtator_dict[i.get('id')]=i


In [96]:
second_filtered_pubtator_json[0]

{'_id': '36895572|None',
 'id': '36895572',
 'infons': {},
 'passages': [{'infons': {'journal': 'Front Immunol;2023; 14 1116224. doi:10.3389/fimmu.2023.1116224',
    'year': '2023',
    'article-id_pmc': 'PMC9989007',
    'type': 'title',
    'authors': 'Di X, Liu S, Xiang L, Jin X, ',
    'section': 'Title'},
   'offset': 0,
   'text': 'Association between the systemic immune-inflammation index and kidney stone: A cross-sectional study of NHANES 2007-2018.',
   'sentences': [],
   'annotations': [{'id': '2',
     'infons': {'identifier': 'MESH:D007249', 'type': 'Disease'},
     'text': 'inflammation',
     'locations': [{'offset': 40, 'length': 12}]},
    {'id': '3',
     'infons': {'identifier': 'MESH:D007669', 'type': 'Disease'},
     'text': 'kidney stone',
     'locations': [{'offset': 63, 'length': 12}]}],
   'relations': []},
  {'infons': {'type': 'abstract', 'section': 'Abstract'},
   'offset': 122,
   'text': 'Background: The incidence rate of kidney stones increased over the 

In [97]:
from nltk.tokenize import sent_tokenize
import datetime
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,rawlength,lock):
    info=i.get('passages')[1]
    abstract=info.get('text')
    sentences=sent_tokenize(abstract)
    # sentenceslist=[]
    positions=[]
    size=len(sentences)
    start=0
    for k,j in enumerate(sentences):
        
        # sentencesdict1={}
        newj=j.translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        spanlist=re.search(newj,abstract,flags=re.M|re.I).span()
        abstract=abstract[spanlist[1]:]
        
        positions.append((start+spanlist[0],start+spanlist[1]))
        start+=spanlist[1]

    middict={'PMID':i.get('id'),'AB':info.get('text'),'SENTENCE':sentences,'positions':positions,'annotations':info.get('annotations'),'offset':info.get('offset')}
    
    lock.acquire()
    rawlength.append(middict)
    lock.release()
    
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    rawlength=manager.list()
    lock=manager.Lock()
    for i in second_filtered_pubtator_json:
        if i.get('passages'):
            p.apply_async(work, args=(i,rawlength,lock))

    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 7995.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 20s


In [98]:
pubtators_list=list(rawlength)

In [99]:
len(pubtators_list)

3400

In [100]:
pubtator_raw=[]
error=[]
error2=[]
for i in pubtators_list:
    for j in i.get('annotations'):
        for k,l in enumerate(i.get('positions')):
            if j.get('locations')[0].get('offset')>=l[0]+i.get('offset') and j.get('locations')[0].get('offset')<l[1]+i.get('offset'):
                pubtator_raw.append({'id':i.get('PMID'),'AB':i.get('AB'), 'type': j.get('infons').get('type'), \
                                     'identifier':j.get('infons').get('identifier'),'text':j.get('text'),'sentence':i.get('SENTENCE')[k],\
                                     'sentenceSize':len(i.get('SENTENCE')),'sentenceIndex':k})
                break
        else:
            error.append(i)
            error2.append(j)
            


In [101]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

In [102]:
len(error)

0

In [103]:
len(error2)

0

In [104]:
# 60169
len(pubtator_raw)

47302

In [105]:
raw_pick_pubtator=[]
for i in reserved_raw_pick:
    if i.get('AB'):
        if pubtator_raw_dict2AB.get(i.get('PMID')):
            i.update({'AB':pubtator_raw_dict2AB.get(i.get('PMID'))})
            raw_pick_pubtator.append(i)

In [106]:
len(pubtator_raw_dict2AB)

3400

In [107]:
len(reserved_raw_pick)

5256

In [108]:
len(raw_pick_pubtator)

3400

In [113]:
for i in raw_pick_pubtator:
    i['addtime']=now_time

In [114]:
collections_pubtator_info.insert_many(raw_pick_pubtator)

In [115]:
len(list(collections_pubtator_info.find()))

173787

# pubtator_for_compare_latest (pubtator_raw)

In [116]:
collections_pubtator_for_compare_latest=db.pubtator_for_compare_latest

pubtator_raw=list(collections_pubtator_for_compare_latest.find({}))

collections_pubtator_for_compare_latest.remove({'addtime':now_time})

In [117]:
for i in pubtator_raw:
    i['addtime']=now_time

In [118]:
collections_pubtator_for_compare_latest.insert_many(pubtator_raw)

In [119]:
# 63016
len(pubtator_raw)

47302

In [120]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [125]:
pickle.dump(raw_pick,open('results/step1/raw_pick.pkl','wb'))
pickle.dump(raw_pick_pubtator,open('results/step1/raw_pick_pubtator.pkl','wb'))
pickle.dump(pubtator_raw,open('results/step1/pubtator_raw.pkl','wb'))


In [122]:
pubtator_raw[0]

{'id': '36891328',
 'AB': 'Out-of-home mobility is fundamental to older people\'s wellbeing and quality of life. Understanding the unmet mobility needs of older people is a necessary starting point for determining how they can be supported to be mobile. This study estimates the extent of unmet mobility needs among older Australians and identifies the characteristics of those most likely to report unmet mobility needs. Analysis was conducted on nationally representative data of 6,685 older Australians drawn from the 2018 Survey of Disability, Aging and Carers conducted by the Australian Bureau of Statistics. Twelve predictor variables from two conceptual frameworks on older people\'s mobility were included in the multiple logistic regression model. Twelve percent (n = 799) of participants had unmet mobility needs, and associated factors significant in multivariable models included being among the "young-old", having a lower income, having lower levels of self-rated health, having a long

In [123]:
now_time

'2023_03_14'

In [ ]:
import os
os.makedirs('results/step1/')
os.makedirs('results/step2/')
os.makedirs('results/step3/')
os.makedirs('results/step4/')

In [126]:
os.makedirs('results/step4/')